In [4]:
!pip install torch

In [2]:
!pip install transformers

  Using cached transformers-4.41.2-py3-none-any.whl.metadata (43 kB)
  Using cached huggingface_hub-0.23.3-py3-none-any.whl.metadata (12 kB)
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------  41.0/42.0 kB 1.9 MB/s eta 0:00:01
     -------------------------------------- 42.0/42.0 kB 675.2 kB/s eta 0:00:00
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 57.6/57.6 kB 3.0 MB/s eta 0:00:00
Using cached transformers-4.41.2-py3-none-any.whl (9.1 MB)
Using cached huggingface_hub-0.23.3-py3-none-any.whl (401 kB)
   ---------------------------------------- 0.0/269.0 kB ? eta -:--:--
   ------------- -------------------------- 92.2/269.0 kB 1.7 MB/s eta 0:00:01
   ------------------------- -------------- 174.1/269.0 kB 1.7 MB/s eta 0:00:01
   -------------------------------------- - 256.0/269.0 kB 1.8 MB/s eta 0:00:01
   ---------------------------------------- 

In [18]:
!pip install numpy==1.23.4

   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB 435.7 kB/s eta 0:00:34
   ---------------------------------------- 0.1/14.6 MB 651.6 kB/s eta 0:00:23
    --------------------------------------- 0.2/14.6 MB 1.2 MB/s eta 0:00:12
   - -------------------------------------- 0.7/14.6 MB 3.3 MB/s eta 0:00:05
   --- ------------------------------------ 1.2/14.6 MB 4.9 MB/s eta 0:00:03
   ---- ----------------------------------- 1.7/14.6 MB 5.6 MB/s eta 0:00:03
   ------ --------------------------------- 2.2/14.6 MB 6.4 MB/s eta 0:00:02
   ------- -------------------------------- 2.7/14.6 MB 7.0 MB/s eta 0:00:02
   --------- ------------------------------ 3.3/14.6 MB 7.6 MB/s eta 0:00:02
   --------- ------------------------------ 3.4/14.6 MB 7.5 MB/s eta 0:00:02
   --------- ------------------------------ 3.6/14.6 MB 6.7 MB/s eta 0:00:02
   ------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.


In [1]:
import torch

torch.cuda.is_available()

True

In [2]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification
import torch

model_ckpt = "indobenchmark/indobert-large-p1"
tokenizer = BertTokenizer.from_pretrained(model_ckpt)

model = BertForSequenceClassification.from_pretrained(model_ckpt, num_labels=8)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

C:\Users\andre\anaconda3\envs\test_env_gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-large-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1

In [3]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    B_Acc = balanced_accuracy_score(labels, preds)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "Balanced Accuracy": B_Acc}

In [4]:
import pandas as pd
df = pd.read_csv("C:/Users/andre/Downloads/SatriaDataScience-main/data_cleaned_aug_no-stemstop.csv", sep=',')
df.describe()

,c_text3,label
count,17160,17160
unique,17071,8
top,tebet eco park hasil pak anies juga gak sih ma...,Politik
freq,4,2971


In [5]:
df.duplicated().sum()

0

In [6]:
df.isnull().sum()

c_text3    0
label      0
dtype: int64

In [7]:
X = list(df['c_text3'])
y = list(df['label'])

In [8]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_encoded= label_encoder.fit_transform(y)

label_mapping = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))
print("Label to index mapping:", label_mapping)

Label to index mapping: {'Demografi': 0, 'Ekonomi': 1, 'Geografi': 2, 'Ideologi': 3, 'Pertahanan dan Keamanan': 4, 'Politik': 5, 'Sosial Budaya': 6, 'Sumber Daya Alam': 7}


In [9]:
#SPLIT THE DATASET
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y_encoded, test_size =0.2, random_state =0)

In [10]:
def tokenize(batch):
    return tokenizer(batch, padding='max_length', truncation=True, max_length=80, return_tensors='pt')

In [11]:
train_encodings = tokenize(X_train)
test_encodings  = tokenize(X_test)

train_encodings

{'input_ids': tensor([[    2, 18006, 30359,  ...,     0,     0,     0],
        [    2, 18006, 30359,  ...,     0,     0,     0],
        [    2,  1369, 19145,  ...,     0,     0,     0],
        ...,
        [    2,  2038, 15994,  ...,     0,     0,     0],
        [    2,   596, 20425,  ...,     0,     0,     0],
        [    2,   511,  6343,  ...,  3281,   626,     3]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]])}

In [12]:
class Dataset(torch.utils.data.Dataset):
  def __init__(self,encodings,labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self,idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item['labels']=torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)

train_dataset = Dataset(train_encodings, y_train)
test_dataset = Dataset(test_encodings, y_test)

In [ ]:
from transformers import Trainer, TrainingArguments
batch_size = 8

logging_steps = len(train_dataset) // batch_size

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs= 3,
    learning_rate = 1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_steps = 500,
    evaluation_strategy ="epoch",
    weight_decay=0.01,
    disable_tqdm=False,
    logging_dir='./logs',
    logging_steps=logging_steps,
    log_level="error"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset= train_dataset,
    eval_dataset = test_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

C:\Users\andre\anaconda3\envs\test_env_gpu\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\andre\AppData\Local\Temp\ipykernel_7788\4091585941.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
C:\Users\andre\anaconda3\envs\test_env_gpu\lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Epoch,Training Loss,Validation Loss


In [ ]:
preds_output = trainer.predict(test_dataset)
preds_output.metrics
y_pred=np.argmax(preds_output.predictions, axis =1)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm= confusion_matrix(y_pred, y_test)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()

In [ ]:
from torch.utils.data import Dataset, DataLoader

df_test = pd.read_csv("C:/Users/andre/Downloads/SatriaDataScience-main/dataset_unlabeled_penyisihan_bdc_2024.csv", sep = ";")
test = list(df_test["Text"])

class TextDataset(Dataset):
    def __init__(self, texts, tokenizer):
        self.texts = texts
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoded = self.tokenizer(text, padding='max_length', truncation=True, return_tensors='pt')
        item = {key: val.squeeze(0) for key, val in encoded.items()}
        return item

# Create dataset and dataloader
df_set = TextDataset(test, tokenizer)
test_dataloader = DataLoader(df_test, batch_size=2)

predictions = trainer.predict(test_dataloader)
pred_labels = predictions.predictions.argmax(-1)


In [ ]:
index_to_label = {v: k for k, v in label_mapping.items()}
string_labels = [index_to_label[label] for label in pred_labels]

jawaban = pd.read_csv("C:/Users/andre/Downloads/SatriaDataScience-main/template_jawaban_penyisihan_bdc_2024.csv", sep = ";")
jawaban["Kelas"] = string_labels
jawaban.to_csv('JawabanBDC.csv', sep=';', index=False,header=True)